In [14]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
import re

In [15]:
data = pd.read_csv('data_estag_ds.tsv', sep='\t')

In [16]:
data.head(10)

,ID,TITLE
0,1041354,Acessório T - Jean Bag For Girls para DS Lite
1,1041782,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...
2,1041834,Carrinho de Bebê para Gêmeos Berço-Passeio - T...
3,1042568,Car Center - Calesita
4,1042584,Donka Trem com Som - Calesita
5,1043696,Interfone HDL AZ-01 Branco
6,1940486,Smartphone LG Optimus L7 II Dual P716 Preto co...
7,2277815,Smartphone Samsung Galaxy S4 Mini Duos Preto c...
8,2961097,Smartphone Samsung Galaxy S5 SM-G900M Branco ...
9,2961099,Smartphone Samsung Galaxy S5 SM-G900M Dourado...


In [17]:
data.shape

(2606, 2)

In [18]:
# Removendo o atributo ID (irrelevante)
ids = data['ID']
data.drop(['ID'], axis=1, inplace=True)

In [19]:
# Inicializando atributos target com 0.0
data['SMARTPHONE'] = np.zeros((len(data)))

# Parte 1: Anotação dos dados
Objetivando a aplicação de um algoritmo supervisionado, foram classificados manualmente os dados entre 1.0 (Smartphone) e 0.0 (Não-Smartphone) e depois filtrados os possíveis casos indesejáveis. Tal tarefa é relativamente simples, pois os intervalos onde há smartphones são facilmente identificáveis. Caso os dados estivessem muito embaralhados e/ou o conjunto de dados fosse muito grande, outras técnicas poderiam ser usadas (i.e. clustering, regex).

In [20]:
# Manualmente selecionando os intervalos onde existem smartphones
data['SMARTPHONE'].iloc[6:39] = 1.0
data['SMARTPHONE'].iloc[2035:] = 1.0

# Nota: Há alguns smartwatches nesse intervalo.
#       Além disso, pode ser que haja algum smartphone que não esteja nesse intervalo.
#       Esses casos serão tratados a seguir

In [21]:
# Casos de Smartwatches classificados como 1.0
for i, example in data[data['SMARTPHONE'] == 1.0]['TITLE'].iteritems():
    if re.findall('watch|rel[ó|o]gio', example.lower()):
        print(i, example)
        data['SMARTPHONE'].iloc[i] = 0.0

2207 Apple Watch Série 3 42mm Cores E Edição  Nike+ Em Estoque
2227 Apple Watch Series 3 42mm Nike + Gps Prova D'água Lacrado
2244 Apple Watch Series 3 42mm Gps Garantia Apple + Brinde
2253 Relógio Samsung Gear S3 Sm-r770 Classic Wifi Bluetooth
2288 Apple Watch Series 3  42mm Nike + Gps Prova D'água Lançameto
2289 Apple Watch Series 3 42mm Nike +  Gps Prova D'água
2298 Apple Watch Series 3 38mm Nike + Gps Prova D'água
2301 Apple Watch Nike+ Séries 3 42mm Lacrado
2302 Apple Watch Nike+ Séries 3 38mm Lacrado Novo
2303 Apple Watch Nike+ Séries 3 38mm Lacrado
2309 Apple Watch Series 3 38mm Nike + Gps Prova D'água Lacrado
2314 Apple Watch Série 3 42mm Gps + 2 Pulseiras Extras
2333 Relógio Samsung S3 Frontier + 2 Pulseiras Extras
2345 Apple Watch Series 3 42mm Lacrado + Brinde
2353 Apple Watch Nike+ Séries 3 42mm Lacrado + Envio Imediato
2380 Apple Watch Series 3 38mm Lacrado Nf + Brinde
2417 Smartwatch Samsung Gear S3 Frontier 1.3 Bluetooth - Original
2597 Relógio Smartwatch Samsung Gear Sm

In [22]:
# Casos de smartphones classificados como 0.0
for i, example in data[data['SMARTPHONE'] == 0.0]['TITLE'].iteritems():
    if re.findall('^smartphone|^celular', example.lower()):
        print(i, example)
        
# Somente 2 casos
data['SMARTPHONE'].iloc[1892:1894] = 1.0

# Nota: Foram encontrados alguns celulares, mas que não são smartphones. Estes foram desconsiderados.

548 Celular Multilaser Up- Branco/Rosa
775 Celular Blu Samba Jr Q53 Prata Dual Chip Silver
1886 Celular Up 2chip Quad Cam Mp3/4 Fm Branco/Rosa
1892 Smartphone Samsung Galaxy J2 Prime TV Dourado com 8GB, Dual chip, Tela 5", TV Digital, 4G, Câmera 8MP, Android 6.0 e Processador Quad Core de 1.4 Ghz
1893 Smartphone Samsung Galaxy J2 Prime TV Preto com 8GB, Dual chip, Tela 5", TV Digital, 4G, Câmera 8MP, Android 6.0 e Processador Quad Core de 1.4 Ghz


In [23]:
data['SMARTPHONE'].value_counts()

0.0    2018
1.0     588
Name: SMARTPHONE, dtype: int64

In [11]:
data['SMARTPHONE'].mean()
# Classes estão desbalanceadas

0.22524942440521872

# Parte 2: Pré-processamento

In [25]:
data['TITLE'] = data['TITLE'].apply(word_tokenize)

In [26]:
data

,TITLE,SMARTPHONE
0,"[Acessório, T, -, Jean, Bag, For, Girls, para,...",0.0
1,"[Carrinho, de, Bebê, Berço-Passeio, -, Pegasus...",0.0
2,"[Carrinho, de, Bebê, para, Gêmeos, Berço-Passe...",0.0
3,"[Car, Center, -, Calesita]",0.0
4,"[Donka, Trem, com, Som, -, Calesita]",0.0
5,"[Interfone, HDL, AZ-01, Branco]",0.0
6,"[Smartphone, LG, Optimus, L7, II, Dual, P716, ...",1.0
7,"[Smartphone, Samsung, Galaxy, S4, Mini, Duos, ...",1.0
8,"[Smartphone, Samsung, Galaxy, S5, SM-G900M, Br...",1.0
9,"[Smartphone, Samsung, Galaxy, S5, SM-G900M, Do...",1.0
